In [1]:
!pip install pygame

In [2]:
import pygame
import random

pygame.init()

# Game settings
window_size = (800, 600)
cell_size = 20
snake_speed = 150

pygame 2.5.0 (SDL 2.28.0, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Colors
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)

In [4]:
# Initialize the game window
window = pygame.display.set_mode(window_size)
pygame.display.set_caption('Snake Game')

In [5]:
# initial positions
snake = [(window_size[0] // 2, window_size[1] // 2)]
snake_direction = (1, 0)
food = (random.randint(0, (window_size[0] - cell_size) // cell_size) * cell_size,
        random.randint(0, (window_size[1] - cell_size) // cell_size) * cell_size)

In [6]:
# Q-learning parameters
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 0.1
q_table = {}

In [14]:
def discretize_state(state):
    snake, food = state
    head_x, head_y = snake[0]
    food_x, food_y = food
    return ((head_x // cell_size, head_y // cell_size), (food_x // cell_size, food_y // cell_size))

In [15]:
def get_best_action(state):
    state = discretize_state(state)
    if state not in q_table:
        q_table[state] = [0, 0, 0, 0]
    if random.random() < exploration_rate:
        return random.randint(0, 3)
    return q_table[state].index(max(q_table[state]))

In [16]:
def update_q_values(state, action, next_state, reward):
    state = discretize_state(state)
    next_state = discretize_state(next_state)
    if state not in q_table:
        q_table[state] = [0, 0, 0, 0]  # Initialize Q-values for the new state
    if next_state not in q_table:
        q_table[next_state] = [0, 0, 0, 0]  # Initialize Q-values for the new next state
    q_table[state][action] = q_table[state][action] + learning_rate * (reward + discount_factor * max(q_table[next_state]) - q_table[state][action])


In [18]:
# Game
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = get_best_action((snake, food))

    keys = pygame.key.get_pressed()
    if action == 0 and snake_direction != (1, 0):  # Left
        snake_direction = (-1, 0)
    elif action == 1 and snake_direction != (-1, 0):  # Right
        snake_direction = (1, 0)
    elif action == 2 and snake_direction != (0, 1):  # Up
        snake_direction = (0, -1)
    elif action == 3 and snake_direction != (0, -1):  # Down
        snake_direction = (0, 1)

    head_x, head_y = snake[0]
    new_head = (head_x + snake_direction[0] * cell_size, head_y + snake_direction[1] * cell_size)
    snake = [new_head] + snake[:-1]

    # Check for collision with food
    if new_head == food:
        food = (random.randint(0, (window_size[0] - cell_size) // cell_size) * cell_size,
                random.randint(0, (window_size[1] - cell_size) // cell_size) * cell_size)
        snake.append(snake[-1])

    # Check for collision with walls or self
    if new_head[0] < 0 or new_head[0] >= window_size[0] or new_head[1] < 0 or new_head[1] >= window_size[1] or new_head in snake[1:]:
        running = False

    # Update Q-values
    reward = 1 if new_head == food else 0
    update_q_values((snake, food), action, (snake[1:] + [new_head], food), reward)


    window.fill(black)

    # Draw the snake
    for segment in snake:
        pygame.draw.rect(window, green, (segment[0], segment[1], cell_size, cell_size))

    # Draw the food
    pygame.draw.rect(window, red, (food[0], food[1], cell_size, cell_size))

    # Update the display
    pygame.display.update()

    # Set the game speed
    clock.tick(snake_speed)



pygame.quit()